# ETL Pipeline for Pre-Processing the Files

#### Importing Python packages 

In [ ]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob 
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [ ]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [ ]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# get total number of rows 
    #print(len(full_data_rows_list))
# check to see what the list of event data rows will look like
    #print(full_data_rows_list)

# Creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [ ]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

# Apache Cassandra 

## The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [ ]:
# A connection to a Cassandra instance
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Creating a Keyspace

In [ ]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity
    WITH REPLICATION =
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)
except Execption as e:
    print(e)

#### Setting the Keyspace

In [ ]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Query 1. The artist, song title and song's length in the music app history heard during sessionId = 338, and itemInSession  = 4

In [ ]:
query = "CREATE TABLE IF NOT EXISTS song_history_by_session"
query = query + "(sessionId int, \
                itemInSession int, \
                artist text, \
                song text, \
                length float, \
                PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [ ]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_history_by_session (sessionId, itemInSession, artist, song, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### SELECT statement to verify that the data have been inserted into each table

In [ ]:
query = "SELECT artist, song, length FROM song_history_by_session WHERE sessionId = 338 AND itemInSession = 4"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row)

### Query 2. name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [ ]:
## Query 2: Give the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

query = "CREATE TABLE IF NOT EXISTS song_history_by_user"
query = query + "( userId int, \
                sessionId int, \ 
                itemInSession int, \
                artist text, \
                firstName text, \
                lastName text, \
                song text, \
                primary key((userId,sessionId),itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [ ]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        user = line[1]+" "+line[4]
        query = "INSERT INTO song_history_by_user (userId, sessionId, itemInSession, artist, firstName, lastName, song)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute( query, ( int(line[10]), int(line[8]), int(line[3]), line[0], line[1], line[4], line[9] ))

#### SELECT statement to verify that the data have been inserted into each table

In [ ]:
query = "SELECT artist, song, firstName, lastName FROM song_history_by_user WHERE userId = 10 AND sessionId = 182"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row)

### Query 3. Every user name (first and last) in the music app history that listened to the song 'All Hands Against His Own'

In [ ]:
query = "CREATE TABLE IF NOT EXISTS song_history_by_song"
query = query + "(song text, \
                userId int, \
                firstName text, \
                lastName text, \
                PRIMARY KEY (song, userId))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [ ]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
## INSERT statements into the `query` variable
        query = "INSERT INTO song_history_by_song (song, userId, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

#### SELECT statement to verify that the data have been inserted into each table

In [ ]:
query = "SELECT song, firstName, lastName FROM song_history_by_song WHERE song = 'All Hands Against His Own'"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row)

### Dropping the tables before closing out the sessions

In [ ]:
session.execute("DROP TABLE IF EXISTS song_history")

session.execute("DROP TABLE IF EXISTS song_history1")

session.execute("DROP TABLE IF EXISTS song_history2")

### Closing the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()